<a href="https://colab.research.google.com/github/talhaanwarch/Offensive-Language-Detection/blob/master/OLID_Classification_FastText_biGRU_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.models import Sequential,Model
from keras.layers import Embedding,CuDNNLSTM,CuDNNGRU,Dense,Dropout,Bidirectional,GlobalMaxPool1D,GlobalAveragePooling1D, SpatialDropout1D,Input,Conv1D,concatenate,GlobalMaxPooling1D,MaxPooling1D,Flatten
from keras.optimizers import RMSprop
from keras.initializers import Constant
from sklearn.utils import class_weight
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
cd /content/drive/My Drive/dataset/OLID

/content/drive/My Drive/dataset/OLID


In [4]:
ls

elmo-model.h5      labels-levelb.csv       testset-levelb.tsv
glove.6B.100d.txt  labels-levelc.csv       testset-levelc.tsv
glove.6B.200d.txt  olid-training-v1.0.tsv
labels-levela.csv  testset-levela.tsv


In [5]:
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import string
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
train=pd.read_csv( 'olid-training-v1.0.tsv',sep="\t")
test=pd.read_csv('testset-levela.tsv',sep="\t")
y_test=pd.read_csv( 'labels-levela.csv',header=None).iloc[:,-1]
#OFF=0 
#NOT=1

In [7]:
y_train=train['subtask_a']
train=train['tweet']
test=test['tweet']
y_train=pd.factorize(y_train)[0]
y_test=pd.factorize(y_test)[0]

import collections
collections.Counter(y_train)

#Counter({0: 4400, 1: 8840})

Counter({0: 4400, 1: 8840})

In [0]:
# Creating the training corpus
stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()
corpus_train = []
for i in train:
    x=i.lower()
    x=x.replace('@user','')
    x=x.replace('@[\w\-]+','')
    #x=x.translate(str.maketrans('', '', string.punctuation))
    x = re.sub('[^A-Za-z]', ' ', x)
    #x=re.sub('\s+',' ',x)
    x=re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',x) #url
    x = [lemmatizer.lemmatize(token) for token in x.split(" ")]
    x = [word for word in x if not word in stop_words]
    x=" ".join(x)
    corpus_train.append(x)    
# Creating the training corpus
corpus_test = []
for i in test:
    x=i.lower()
    x=x.replace('@user','')
    x=x.replace('@[\w\-]+','')
    #x=x.translate(str.maketrans('', '', string.punctuation))
    x = re.sub('[^A-Za-z]', ' ', x)
    #x=re.sub('\s+',' ',x)
    x=re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',x) #url
    x = [lemmatizer.lemmatize(token) for token in x.split(" ")]
    x = [word for word in x if not word in stop_words]
    x=" ".join(x)
    corpus_test.append(x) 

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 1000 #frequency of words to be kept
max_len = 200 #MAX_SEQUENCE_LENGTH 

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(corpus_train)
sequences = tokenize.texts_to_sequences(corpus_train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [0]:
from urllib.request import urlopen
import gzip

# get the vectors
file = gzip.open(urlopen('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ro.300.vec.gz'))

In [0]:
vocab_and_vectors = {}
# put words as dict indexes and vectors as words values
for line in file:
  values = line.split()
  word = values [0].decode('utf-8')
  vector = np.asarray(values[1:], dtype='float32')
  vocab_and_vectors[word] = vector

In [0]:
num_words=len(word_index) + 1
embedding_dim=300

In [0]:
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
  embedding_vector = vocab_and_vectors.get(word)
  # words that cannot be found will be set to 0
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [141]:
inp=Input(shape=(max_len, ))
x=Embedding(num_words,embedding_dim,weights=[embedding_matrix],trainable=False)(inp)
x = SpatialDropout1D(0.2)(x)
x=Bidirectional(CuDNNGRU(100, return_sequences = True))(x)
x=CuDNNGRU(100, return_sequences = True)(x)

#x=GlobalMaxPool1D()(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
conc = concatenate([avg_pool, max_pool])
x=Dense(50, activation="relu")(conc)
x=Dense(30, activation="relu")(x)

x=Dropout(0.05)(x)
out=Dense(1, activation="sigmoid")(x)

model=Model(inp,out)
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 200, 300)     4988700     input_3[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_3 (SpatialDro (None, 200, 300)     0           embedding_3[0][0]                
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 200, 200)     241200      spatial_dropout1d_3[0][0]        
____________________________________________________________________________________________

In [142]:

class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))
model.fit(sequences_matrix,y_train,batch_size=128,epochs=5,verbose=2,class_weight=class_weights)

Epoch 1/5
 - 7s - loss: 0.6568 - acc: 0.6036
Epoch 2/5
 - 6s - loss: 0.6144 - acc: 0.6820
Epoch 3/5
 - 6s - loss: 0.5927 - acc: 0.7076
Epoch 4/5
 - 6s - loss: 0.5822 - acc: 0.7200
Epoch 5/5
 - 6s - loss: 0.5751 - acc: 0.7221


In [143]:
test_sequences = tokenize.texts_to_sequences(corpus_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
print(model.evaluate(test_sequences_matrix,y_test))

860/860 [==============================] - 1s 709us/step
[0.527933483345564, 0.7837209302325582]


In [144]:
from sklearn.metrics import classification_report

y_pred = model.predict(test_sequences_matrix, batch_size=128, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_test, y_pred))


860/860 [==============================] - 0s 378us/step
              precision    recall  f1-score   support

           0       0.63      0.54      0.58       240
           1       0.83      0.88      0.85       620

    accuracy                           0.78       860
   macro avg       0.73      0.71      0.72       860
weighted avg       0.78      0.78      0.78       860



In [0]:
import keras
keras.backend.clear_session()

In [0]:
#